In [1]:
!pip install galois
!pip install scipy

/bin/bash: line 1: /home/michele/Desktop/Tirocinio_giusto/Tirocinio/.venv/bin/pip: cannot execute: required file not found
/bin/bash: line 1: /home/michele/Desktop/Tirocinio_giusto/Tirocinio/.venv/bin/pip: cannot execute: required file not found


In [1]:
import galois
import numpy as np
import time
import os
from scipy import stats

In [2]:
#definisco un campo di Galois come nell'esempio 
field = galois.GF(5)

In [3]:
#definisco una matrice k x (n-k)
k = 3 #le righe
n_k = 4 #le colonne

#N.B: Less usa q = dimensione campo: 127; k = 126; n = 252 visto da pdf myLESS_CF pagina 38 di 45

In [4]:
#questa è con l'esempio di 4.4
A = field([[2, 3, 1, 1],
           [2, 1, 1, 1],
           [1, 0, 3, 4]])

In [55]:
A = field(np.random.randint(0, field.order, size=(k, n_k)))
print(A)

[[3 4 1 0]
 [3 0 2 3]
 [3 3 1 2]]


In [5]:
matrix_bin1 = np.array([
    [0, 1, 0],
    [1, 0, 0],
    [0, 0, 1]
])

matrix_bin1 = field(matrix_bin1)

matrix_bin2 = np.array([
    [0, 0, 0, 1],
    [0, 1, 0, 0],
    [1, 0, 0, 0],
    [0, 0, 1, 0]
])

matrix_bin2 = field(matrix_bin2)

matrix_diag = np.array([
    [4, 0, 0],
    [0, 1, 0],
    [0, 0, 3]
])

matrix_diag = field(matrix_diag)

In [6]:
A_primo = field(matrix_bin1 @ matrix_diag @ A @ matrix_bin2)
print(A_primo)

[[1 1 1 2]
 [4 2 4 3]
 [4 0 2 3]]


In [7]:
#number of iteration for Montecarlo simulatiom
num_iterazioni = 2500

In [9]:
def has_duplicate_rows(matrix):
    _, counts = np.unique(matrix, axis=0, return_counts=True)
    return np.any(counts > 1)

#initial matrix x matrix with s or s^(-1)
def make_matrix_xS(matrix, error_counter, field_order):
    matrix_S = field(np.zeros((k, k), dtype = int))
    for i, row in enumerate(matrix):
        print("iter:", i)
        #print("Type row: ", type(row)) IS GALOIS (CONFIRMED)
        #bp.bincout count frequency of element in row
        s = np.sum(row)
        print(s)
        s_prime = np.sum(row ** (field_order-2))
        print(s_prime)
        if s == 0 and s_prime == 0:
            print("esco")
            return 0
        elif s != 0:
            matrix_S[i,i] = np.reciprocal(s)
            print(type(matrix_S))
        else:
            matrix_S[i,i] = s_prime
    
    #print("Matrix_S:", matrix_S)
    matrix = matrix_S @ matrix
    print(type(matrix), matrix)
    return matrix

# order by count number of 0, 1, ..., q in a vector (dictionary). A multiset1 < multiset2 if number of 0 > number of 0 of 
# second multiset. If the number of 0 is the same check number of 1 and so on
def order_row(matrix, error_counter, field_order, max_column):  
    start = time.perf_counter()#this is for make file csv dati_differences_upd2 and only count/ord
    vector_of_counts = np.zeros((matrix.shape[0], field.order + 1), dtype = int)
    n_colonne = matrix.shape[1]  
    #start = time.perf_counter()
    
    #for row in matrix:
    for i, row in enumerate(matrix):
        #bp.bincout count frequency of element in row
        vector_of_counts[i] = np.bincount(row, minlength = field.order + 1)
    end = time.perf_counter() #these 2 for file only count/ord
    duration = (end - start) * 1000
    #vector_of_counts = np.array(vector_of_counts)
    if has_duplicate_rows(vector_of_counts):
        error_counter += 1
        #print(vector_of_counts)
        print("Duplicated rows all columns",error_counter)
        return [], 0, 0, [], 0, 0, error_counter
    
    sorted_index = np.lexsort([vector_of_counts[:, i] for i in range(n_colonne, -1, -1)])[::-1]
    #end = time.perf_counter() #these 2 for make file csv dati_differences_upd2
    #duration = (end - start) * 1000    
    return matrix[sorted_index], duration, 0, [], 0, 0, error_counter
    

# order column as always
def sort_by_column(matrix, rows, num_swap_columns):
    print("Funzione ordina per colonna")
    print(type(matrix), matrix)
    #tengo posizioni originali per confrontarle con quelle cambiate e quindi vedere quanti swap di riga ho fatto
    initial_position = np.arange(matrix.shape[1])
    
    # Trova gli indici di ordinamento usando np.lexsort sulle righe, se c'è conflitto passa a el seconda riga
    sorting_indices = np.lexsort([matrix[row, :] for row in rows[::-1]])
        
    # Conta gli swap solo quando la posizione delle colonne cambia
    for i, index in enumerate(sorting_indices):
        if index != initial_position[i]:
            num_swap_columns += 1
    
    #riordina la matrice in base agli indici trovati
    sorted_matrix = matrix[:, sorting_indices]
    return sorted_matrix, num_swap_columns

def algorithm_test(matrix1, matrix2):
    row1_sort = order_row(matrix1, 0, field.order, 4)[0]
    print("Ordinato\n", row1_sort)
    matrix1_sorted = sort_by_column(row1_sort, range(k), 0)
    print(matrix1_sorted)

    row2_sort = order_row(matrix2, 0, field.order, 4)[0]
    print(row2_sort)
    matrix2_sorted = sort_by_column(row2_sort, range(k), 0)
    print(matrix2_sorted)

    #[0] because retur an array with 2 elements
    return np.array_equal(matrix1_sorted[0], matrix2_sorted[0])



In [10]:
A = make_matrix_xS(A, 0, field.order)
print(A)
A_primo = make_matrix_xS(A_primo, 0, field.order)
algorithm_test(A, A_primo)

iter: 0
1
4
<class 'galois.GF(5)'>
iter: 1
0
1
iter: 2
1
1
<class 'galois.GF(5)'>
<class 'galois.GF(5)'> [[1 4 3 3]
 [2 1 1 1]
 [2 0 1 3]]
[[1 4 3 3]
 [2 1 1 1]
 [2 0 1 3]]
iter: 0
0
1
iter: 1
1
4
<class 'galois.GF(5)'>
iter: 2
1
1
<class 'galois.GF(5)'>
<class 'galois.GF(5)'> [[1 1 1 2]
 [3 4 3 1]
 [1 0 3 2]]
Ordinato
 [[2 0 1 3]
 [2 1 1 1]
 [1 4 3 3]]
Funzione ordina per colonna
<class 'galois.GF(5)'> [[2 0 1 3]
 [2 1 1 1]
 [1 4 3 3]]
(GF([[0, 1, 2, 3],
    [1, 1, 2, 1],
    [4, 3, 1, 3]], order=5), 3)
[[1 0 3 2]
 [1 1 1 2]
 [3 4 3 1]]
Funzione ordina per colonna
<class 'galois.GF(5)'> [[1 0 3 2]
 [1 1 1 2]
 [3 4 3 1]]
(GF([[0, 1, 2, 3],
    [1, 1, 2, 1],
    [4, 3, 1, 3]], order=5), 4)


True